In [1]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [8]:
data = sc.textFile("file:///home/cloudera/Downloads/ml-100k/u.data")
#data = sc.textFile("data/mllib/als/test.data")

In [19]:
data.take(3)

['196\t242\t3\t881250949', '186\t302\t3\t891717742', '22\t377\t1\t878887116']

In [20]:
ratings = data.map(lambda l: l.split('\t'))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [21]:
ratings.take(3)

[Rating(user=196, product=242, rating=3.0),
 Rating(user=186, product=302, rating=3.0),
 Rating(user=22, product=377, rating=1.0)]

In [22]:
training_RDD, validation_RDD = ratings.randomSplit([7, 3], seed=0)

In [23]:
from pyspark.mllib.recommendation import ALS
import math

In [24]:
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))

In [28]:
import time
t_start=time.clock()
seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank
t_end=time.clock()
print ('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.9339855426263448
For rank 8 the RMSE is 0.9368369419106055
For rank 12 the RMSE is 0.9341046056997685
The best model was trained with rank 4


In [30]:
CPUt=t_end-t_start
print ('CPU time is %s' %(CPUt))

CPU time is 0.23000000000000043
